# Extraindo empresas

## Extraindo todas empresas do Brazil da categoria BESST

In [9]:
import requests
import yfinance as yf
import pandas as pd
import os

# --- PASSO 1: DEFINIR OS SETORES-MÃE DE INTERESSE ---
# A regra para buscar empresas destes setores permanece.
setores_alvo = [
    "Finance",
    "Utilities",
    "Communications"
]

# --- FUNÇÃO PRINCIPAL COM A LÓGICA ATUALIZADA ---
def scanner_sem_filtro_marketcap(setores):
    """
    Busca todas as empresas dos setores-alvo, sem descartar ativos
    com base no valor de mercado (market cap).
    """
    print("Iniciando scanner (sem filtro de market cap)...")
    try:
        url = "https://brapi.dev/api/quote/list"
        lista_completa_brapi = requests.get(url).json().get('stocks', [])
    except Exception as e:
        print(f"❌ Erro fatal ao buscar a lista da brapi: {e}")
        return {}

    # Etapa 1: Coleta Ampla
    print("Mapeando todos os ativos dos setores alvo...")
    tickers_para_analisar = {}
    for stock in lista_completa_brapi:
        setor_brapi = stock.get('sector')
        if setor_brapi in setores:
            ticker = f"{stock.get('stock')}.SA"
            tickers_para_analisar[ticker] = setor_brapi
    
    total = len(tickers_para_analisar)
    print(f"✅ Mapeamento concluído. {total} ativos serão analisados em detalhe.")

    # Etapa 2: Enriquecimento de Dados
    dados_finais = {}
    print("\nIniciando coleta detalhada de dados no yfinance...")
    for i, (ticker, setor_brapi) in enumerate(tickers_para_analisar.items()):
        try:
            print(f"Analisando [{i+1}/{total}]: {ticker}...")

            stock = yf.Ticker(ticker)
            info = stock.info
            
            # Filtro de País permanece
            if info.get("country") != "Brazil":
                print(f"  -> DESCARTADO: {ticker} não é do Brasil.")
                continue
            
            # <<< REGRA 7 REMOVIDA DAQUI >>>
            # O filtro de Market Cap foi completamente removido conforme solicitado.
            
            # Coleta dos dados
            dados_finais[ticker] = {
                "Empresa": info.get("longName", "N/A"),
                "Setor (brapi)": setor_brapi,
                "Subsetor (yfinance)": info.get("industry", "N/A"),
                "País": info.get("country", "N/A"),
                "Market Cap": info.get("marketCap"), # Coleta o valor, mesmo que seja nulo
            }
            print(f"  -> ✅ INCLUÍDO: {ticker}")

        except Exception as e:
            print(f"  -> ERRO: Falha ao processar {ticker}. Erro: {e}")
            continue
                
    return dados_finais

# --- EXECUÇÃO E APRESENTAÇÃO DOS RESULTADOS ---
resultados = scanner_sem_filtro_marketcap(setores_alvo)

if resultados:
    print("\n✅ Análise finalizada.")
    df_resultado = pd.DataFrame.from_dict(resultados, orient="index")
    
    df_resultado.reset_index(inplace=True)
    df_resultado.rename(columns={'index': 'Ticker'}, inplace=True)
    
    # A lógica de formatação e ordenação lida com valores nulos de Market Cap.
    # Empresas sem Market Cap serão convertidas para NaN e listadas no final da ordenação.
    if "Market Cap" in df_resultado.columns:
         df_resultado["Market Cap Num"] = pd.to_numeric(df_resultado["Market Cap"], errors='coerce')
         df_resultado["Market Cap"] = df_resultado["Market Cap Num"].apply(
             lambda x: f"R$ {x/1_000_000_000:.2f} Bi" if pd.notna(x) and x > 0 else "N/A"
         )

    # Ordena pelo Setor principal e depois pelo Market Cap (empresas sem valor por último)
    df_ordenado = df_resultado.sort_values(by=['Setor (brapi)', 'Market Cap Num'], ascending=[True, False])
    
    colunas_finais = ['Ticker', 'Empresa', 'Setor (brapi)', 'Subsetor (yfinance)', 'País', 'Market Cap']
    df_final = df_ordenado[colunas_finais]
    
    print("\n--- Relatório Amplo com Subsetores (Inclusivo) ---")
    print(df_final.to_string(index=False))

    # --- SALVANDO O ARQUIVO NO CAMINHO ESPECÍFICO ---
    try:
        output_folder = r"E:\Github\finance-manager\datasets"
        output_filename = "scanner_amplo_inclusivo_b3.xlsx" # Nome atualizado para refletir a nova regra
        full_path = os.path.join(output_folder, output_filename)
        
        os.makedirs(output_folder, exist_ok=True)
        
        df_final.to_excel(full_path, index=False)
        print(f"\n✅ Resultados salvos com sucesso no caminho: {full_path}")
    except Exception as e:
        print(f"\n❌ Erro ao salvar o arquivo Excel: {e}")
else:
    print("\nNenhuma empresa foi encontrada para os critérios especificados.")

Iniciando scanner (sem filtro de market cap)...
Mapeando todos os ativos dos setores alvo...
✅ Mapeamento concluído. 358 ativos serão analisados em detalhe.

Iniciando coleta detalhada de dados no yfinance...
Analisando [1/358]: BBDC4.SA...
  -> ✅ INCLUÍDO: BBDC4.SA
Analisando [2/358]: BBAS3.SA...
  -> ✅ INCLUÍDO: BBAS3.SA
Analisando [3/358]: VAMO3.SA...
  -> ✅ INCLUÍDO: VAMO3.SA
Analisando [4/358]: B3SA3.SA...
  -> ✅ INCLUÍDO: B3SA3.SA
Analisando [5/358]: ITSA4.SA...
  -> ✅ INCLUÍDO: ITSA4.SA
Analisando [6/358]: CSAN3.SA...
  -> ✅ INCLUÍDO: CSAN3.SA
Analisando [7/358]: CPLE6.SA...
  -> ✅ INCLUÍDO: CPLE6.SA
Analisando [8/358]: RENT3.SA...
  -> ✅ INCLUÍDO: RENT3.SA
Analisando [9/358]: ITUB4.SA...
  -> ✅ INCLUÍDO: ITUB4.SA
Analisando [10/358]: MOVI3.SA...
  -> ✅ INCLUÍDO: MOVI3.SA
Analisando [11/358]: CMIG4.SA...
  -> ✅ INCLUÍDO: CMIG4.SA
Analisando [12/358]: CPLE3.SA...
  -> ✅ INCLUÍDO: CPLE3.SA
Analisando [13/358]: BBDC3.SA...
  -> ✅ INCLUÍDO: BBDC3.SA
Analisando [14/358]: ENEV3.SA...


## Scanner completo

In [10]:
import requests
import yfinance as yf
import pandas as pd
import os

# --- FUNÇÃO PRINCIPAL SEM FILTROS ---
def scanner_completo():
    """
    Busca TODOS os ativos da API brapi e coleta suas informações detalhadas
    no yfinance, sem aplicar filtros de setor, país ou market cap.
    """
    print("Iniciando scanner COMPLETO da B3 (sem filtros)...")
    print("AVISO: Este processo pode ser demorado.")
    try:
        url = "https://brapi.dev/api/quote/list"
        lista_completa_brapi = requests.get(url).json().get('stocks', [])
    except Exception as e:
        print(f"❌ Erro fatal ao buscar a lista da brapi: {e}")
        return {}

    # Etapa 1: Mapear TODOS os tickers para seus setores
    print("Mapeando todos os ativos encontrados...")
    tickers_para_analisar = {}
    for stock in lista_completa_brapi:
        ticker = f"{stock.get('stock')}.SA"
        # Usamos 'N/A' se o setor não for informado na brapi
        setor_brapi = stock.get('sector', 'N/A')
        tickers_para_analisar[ticker] = setor_brapi
    
    total = len(tickers_para_analisar)
    print(f"✅ Mapeamento concluído. {total} ativos serão analisados em detalhe.")

    # Etapa 2: Coleta de Dados Detalhada
    dados_finais = {}
    print("\nIniciando coleta detalhada de dados no yfinance...")
    for i, (ticker, setor_brapi) in enumerate(tickers_para_analisar.items()):
        try:
            # Imprime o progresso a cada 10 tickers para não poluir a tela
            if (i + 1) % 10 == 0:
                print(f"Analisando [{i+1}/{total}]: {ticker}...")

            stock = yf.Ticker(ticker)
            info = stock.info
            
            # Todos os filtros foram removidos conforme solicitado.
            # Apenas o try/except permanece para lidar com erros de tickers inválidos.

            dados_finais[ticker] = {
                "Empresa": info.get("longName", "N/A"),
                "Setor (brapi)": setor_brapi,
                "Subsetor (yfinance)": info.get("industry", "N/A"),
                "País": info.get("country", "N/A"),
                "Market Cap": info.get("marketCap"),
                # <<< NOVA COLUNA ADICIONADA AQUI >>>
                "Logo": info.get("logo_url", "N/A"),
            }
        except Exception:
            # Ignora silenciosamente qualquer erro para um ticker específico e continua
            continue
                
    return dados_finais

# --- EXECUÇÃO E APRESENTAÇÃO DOS RESULTADOS ---
resultados = scanner_completo()

if resultados:
    print("\n✅ Análise completa finalizada.")
    df_resultado = pd.DataFrame.from_dict(resultados, orient="index")
    
    df_resultado.reset_index(inplace=True)
    df_resultado.rename(columns={'index': 'Ticker'}, inplace=True)
    
    if "Market Cap" in df_resultado.columns:
         df_resultado["Market Cap Num"] = pd.to_numeric(df_resultado["Market Cap"], errors='coerce')
         df_resultado["Market Cap"] = df_resultado["Market Cap Num"].apply(
             lambda x: f"R$ {x/1_000_000_000:.2f} Bi" if pd.notna(x) and x > 0 else "N/A"
         )

    # Ordena pelo Market Cap do maior para o menor como critério principal
    df_ordenado = df_resultado.sort_values(by=['Market Cap Num'], ascending=False)
    
    # Define a ordem final das colunas para o relatório
    colunas_finais = ['Ticker', 'Empresa', 'Setor (brapi)', 'Subsetor (yfinance)', 'País', 'Market Cap', 'Logo']
    df_final = df_ordenado[colunas_finais]
    
    print("\n--- Relatório Completo da B3 ---")
    print(df_final.to_string(index=False))

    # --- SALVANDO O ARQUIVO NO CAMINHO ESPECÍFICO ---
    try:
        output_folder = r"E:\Github\finance-manager\datasets"
        output_filename = "scanner_completo.xlsx" # Novo nome do arquivo
        full_path = os.path.join(output_folder, output_filename)
        
        os.makedirs(output_folder, exist_ok=True)
        
        df_final.to_excel(full_path, index=False)
        print(f"\n✅ Resultados salvos com sucesso no caminho: {full_path}")
    except Exception as e:
        print(f"\n❌ Erro ao salvar o arquivo Excel: {e}")
else:
    print("\nNenhuma empresa foi encontrada ou processada com sucesso.")

Iniciando scanner COMPLETO da B3 (sem filtros)...
AVISO: Este processo pode ser demorado.
Mapeando todos os ativos encontrados...
✅ Mapeamento concluído. 1705 ativos serão analisados em detalhe.

Iniciando coleta detalhada de dados no yfinance...
Analisando [10/1705]: PETR4.SA...
Analisando [20/1705]: LREN3.SA...
Analisando [30/1705]: PRIO3.SA...
Analisando [40/1705]: BBDC3.SA...
Analisando [50/1705]: EQTL3.SA...
Analisando [60/1705]: HAPV3.SA...
Analisando [70/1705]: ELET3.SA...
Analisando [80/1705]: VIVA3.SA...
Analisando [90/1705]: YDUQ3.SA...
Analisando [100/1705]: FLRY3.SA...
Analisando [110/1705]: SAPR11.SA...
Analisando [120/1705]: LIGT3.SA...
Analisando [130/1705]: AZEV3.SA...
Analisando [140/1705]: EZTC3.SA...
Analisando [150/1705]: GRND3.SA...
Analisando [160/1705]: ELET6.SA...
Analisando [170/1705]: SHUL4.SA...
Analisando [180/1705]: CSED3.SA...
Analisando [190/1705]: AURA33.SA...
Analisando [200/1705]: NASD11.SA...
Analisando [210/1705]: MELI34.SA...
Analisando [220/1705]: 

In [24]:

df_dados = pd.read_excel(r'E:\Github\finance-manager\datasets\scanner_completo.xlsx')


df_dados = df_dados[df_dados['Ticker'] == 'TAEE4.SA']

# Filter rows where 'Setor (brapi)' is one of the specified values
df_dados = df_dados[df_dados['Setor (brapi)'].isin(["Finance", "Utilities", "Communications", "Industrial Services"])]

# Filter rows where País equals 'Brazil'
# df_dados = df_dados[df_dados['País'] == 'Brazil']


print(df_dados)


       Ticker                                        Empresa  \
408  TAEE4.SA  Transmissora Aliança de Energia Elétrica S.A.   

           Setor (brapi)             Subsetor (yfinance)    País   Market Cap  \
408  Industrial Services  Utilities - Regulated Electric  Brazil  R$ 11.87 Bi   

     Logo  
408   NaN  


# Extração filtrada

In [25]:
import requests
import yfinance as yf
import pandas as pd
import os

# --- PASSO 1: REATIVANDO O FILTRO DE SETORES ---
# A busca volta a ser focada nestes setores-mãe.
setores_alvo = [
    "Finance",
    "Utilities",
    "Communications",
    "Industrial Services"
]

# --- FUNÇÃO PRINCIPAL COM A LÓGICA FINAL ---
def scanner_final(setores):
    """
    Busca empresas dos setores-alvo, filtrando por país e corrigindo
    a busca pelo logo, mas sem filtrar por market cap.
    """
    print("Iniciando scanner final focado...")
    try:
        url = "https://brapi.dev/api/quote/list"
        lista_completa_brapi = requests.get(url).json().get('stocks', [])
    except Exception as e:
        print(f"❌ Erro fatal ao buscar a lista da brapi: {e}")
        return {}

    # Etapa 1: Mapear tickers, seus setores E O LOGO da brapi
    print("Mapeando ativos dos setores alvo...")
    tickers_para_analisar = {}
    for stock in lista_completa_brapi:
        setor_brapi = stock.get('sector')
        # Reativa o filtro de setor
        if setor_brapi in setores:
            ticker = f"{stock.get('stock')}.SA"
            # <<< CORREÇÃO DO LOGO APLICADA AQUI >>>
            # Armazenamos o setor e o logo da brapi para uso posterior
            tickers_para_analisar[ticker] = {
                'setor': setor_brapi,
                'logo': stock.get('logo') 
            }
    
    total = len(tickers_para_analisar)
    print(f"✅ Mapeamento concluído. {total} ativos serão analisados em detalhe.")

    # Etapa 2: Coleta de Dados Detalhada
    dados_finais = {}
    print("\nIniciando coleta detalhada de dados no yfinance...")
    for i, (ticker, brapi_data) in enumerate(tickers_para_analisar.items()):
        try:
            print(f"Analisando [{i+1}/{total}]: {ticker}...")

            stock = yf.Ticker(ticker)
            info = stock.info
            
            # <<< FILTRO DE PAÍS REATIVADO >>>
            if info.get("country") != "Brazil":
                print(f"  -> DESCARTADO: {ticker} não é do Brasil.")
                continue
            
            # Filtro de Market Cap continua desativado para garantir a inclusão da Taesa

            dados_finais[ticker] = {
                "Empresa": info.get("longName", "N/A"),
                "Setor (brapi)": brapi_data['setor'],
                "Subsetor (yfinance)": info.get("industry", "N/A"),
                "País": info.get("country", "N/A"),
                "Market Cap": info.get("marketCap"),
                "Tipo": info.get("type"),
                "Logo": brapi_data['logo'],
            }
            print(f"  -> ✅ INCLUÍDO: {ticker}")

        except Exception as e:
            print(f"  -> ERRO: Falha ao processar {ticker}. Erro: {e}")
            continue
                
    return dados_finais

# --- EXECUÇÃO E APRESENTAÇÃO DOS RESULTADOS ---
resultados = scanner_final(setores_alvo)

if resultados:
    print("\n✅ Análise finalizada.")
    df_resultado = pd.DataFrame.from_dict(resultados, orient="index")
    
    df_resultado.reset_index(inplace=True)
    df_resultado.rename(columns={'index': 'Ticker'}, inplace=True)
    
    if "Market Cap" in df_resultado.columns:
         df_resultado["Market Cap Num"] = pd.to_numeric(df_resultado["Market Cap"], errors='coerce')
         df_resultado["Market Cap"] = df_resultado["Market Cap Num"].apply(
             lambda x: f"R$ {x/1_000_000_000:.2f} Bi" if pd.notna(x) and x > 0 else "N/A"
         )

    df_ordenado = df_resultado.sort_values(by=['Setor (brapi)', 'Market Cap Num'], ascending=[True, False])
    
    colunas_finais = ['Ticker', 'Empresa', 'Setor (brapi)', 'Subsetor (yfinance)', 'País', 'Market Cap', 'Logo']
    df_final = df_ordenado[colunas_finais]
    
    print("\n--- Relatório Filtrado e Inclusivo ---")
    print(df_final.to_string(index=False))

    # --- SALVANDO O ARQUIVO NO CAMINHO ESPECÍFICO ---
    try:
        output_folder = r"E:\Github\finance-manager\datasets"
        output_filename = "scanner_filtrado_final.xlsx" # Novo nome
        full_path = os.path.join(output_folder, output_filename)
        
        os.makedirs(output_folder, exist_ok=True)
        
        df_final.to_excel(full_path, index=False)
        print(f"\n✅ Resultados salvos com sucesso no caminho: {full_path}")
    except Exception as e:
        print(f"\n❌ Erro ao salvar o arquivo Excel: {e}")
else:
    print("\nNenhuma empresa foi encontrada para os critérios especificados.")

Iniciando scanner final focado...
Mapeando ativos dos setores alvo...
✅ Mapeamento concluído. 389 ativos serão analisados em detalhe.

Iniciando coleta detalhada de dados no yfinance...
Analisando [1/389]: BBDC4.SA...
  -> ✅ INCLUÍDO: BBDC4.SA
Analisando [2/389]: BBAS3.SA...
  -> ✅ INCLUÍDO: BBAS3.SA
Analisando [3/389]: VAMO3.SA...
  -> ✅ INCLUÍDO: VAMO3.SA
Analisando [4/389]: B3SA3.SA...
  -> ✅ INCLUÍDO: B3SA3.SA
Analisando [5/389]: ITSA4.SA...
  -> ✅ INCLUÍDO: ITSA4.SA
Analisando [6/389]: CSAN3.SA...
  -> ✅ INCLUÍDO: CSAN3.SA
Analisando [7/389]: CPLE6.SA...
  -> ✅ INCLUÍDO: CPLE6.SA
Analisando [8/389]: RENT3.SA...
  -> ✅ INCLUÍDO: RENT3.SA
Analisando [9/389]: ITUB4.SA...
  -> ✅ INCLUÍDO: ITUB4.SA
Analisando [10/389]: MOVI3.SA...
  -> ✅ INCLUÍDO: MOVI3.SA
Analisando [11/389]: CMIG4.SA...
  -> ✅ INCLUÍDO: CMIG4.SA
Analisando [12/389]: CPLE3.SA...
  -> ✅ INCLUÍDO: CPLE3.SA
Analisando [13/389]: BBDC3.SA...
  -> ✅ INCLUÍDO: BBDC3.SA
Analisando [14/389]: ENEV3.SA...
  -> ✅ INCLUÍDO: ENEV3.

Alteraceso v2

In [26]:
import requests
import yfinance as yf
import pandas as pd
import os

# --- PASSO 1: CONFIGURAÇÃO DOS SETORES-ALVO ---
# A busca é focada nestes setores, incluindo sua adição.
setores_alvo = [
    "Finance",
    "Utilities",
    "Communications",
    "Industrial Services" # Adicionado conforme sua modificação
]

# --- FUNÇÃO PRINCIPAL COM A LÓGICA FINAL ---
def scanner_final_com_tipo(setores):
    """
    Busca apenas AÇÕES (type='stock') dos setores-alvo, filtrando por 
    país e incluindo a coluna 'Tipo' no resultado.
    """
    print("Iniciando scanner final focado (filtrando por tipo 'stock')...")
    try:
        url = "https://brapi.dev/api/quote/list"
        lista_completa_brapi = requests.get(url).json().get('stocks', [])
    except Exception as e:
        print(f"❌ Erro fatal ao buscar a lista da brapi: {e}")
        return {}

    # Etapa 1: Mapear tickers, setores, logos e tipos da brapi
    print("Mapeando ativos dos setores alvo...")
    tickers_para_analisar = {}
    for stock in lista_completa_brapi:
        setor_brapi = stock.get('sector')
        tipo_ativo = stock.get('type')

        # <<< NOVO FILTRO DE TIPO ADICIONADO AQUI >>>
        # Agora o script verifica o setor E se o tipo é 'stock'
        if setor_brapi in setores and tipo_ativo == 'stock':
            ticker = f"{stock.get('stock')}.SA"
            
            # Armazenamos todos os dados que vêm da brapi
            tickers_para_analisar[ticker] = {
                'setor': setor_brapi,
                'logo': stock.get('logo'),
                'tipo': tipo_ativo # Armazena o tipo para a coluna final
            }
    
    total = len(tickers_para_analisar)
    print(f"✅ Mapeamento concluído. {total} ativos do tipo 'stock' serão analisados.")

    # Etapa 2: Coleta de Dados Detalhada
    dados_finais = {}
    print("\nIniciando coleta detalhada de dados no yfinance...")
    for i, (ticker, brapi_data) in enumerate(tickers_para_analisar.items()):
        try:
            print(f"Analisando [{i+1}/{total}]: {ticker}...")
            stock = yf.Ticker(ticker)
            info = stock.info
            
            if info.get("country") != "Brazil":
                print(f"  -> DESCARTADO: {ticker} não é do Brasil.")
                continue
            
            # Coleta dos dados, incluindo a nova coluna 'Tipo'
            dados_finais[ticker] = {
                "Empresa": info.get("longName", "N/A"),
                "Setor (brapi)": brapi_data['setor'],
                "Subsetor (yfinance)": info.get("industry", "N/A"),
                "País": info.get("country", "N/A"),
                # <<< NOVA COLUNA 'TIPO' INCLUÍDA AQUI >>>
                "Tipo": brapi_data['tipo'],
                "Market Cap": info.get("marketCap"),
                "Logo": brapi_data['logo'],
            }
            print(f"  -> ✅ INCLUÍDO: {ticker}")

        except Exception as e:
            print(f"  -> ERRO: Falha ao processar {ticker}. Erro: {e}")
            continue
                
    return dados_finais

# --- EXECUÇÃO E APRESENTAÇÃO DOS RESULTADOS ---
resultados = scanner_final_com_tipo(setores_alvo)

if resultados:
    print("\n✅ Análise finalizada.")
    df_resultado = pd.DataFrame.from_dict(resultados, orient="index")
    
    df_resultado.reset_index(inplace=True)
    df_resultado.rename(columns={'index': 'Ticker'}, inplace=True)
    
    if "Market Cap" in df_resultado.columns:
         df_resultado["Market Cap Num"] = pd.to_numeric(df_resultado["Market Cap"], errors='coerce')
         df_resultado["Market Cap"] = df_resultado["Market Cap Num"].apply(
             lambda x: f"R$ {x/1_000_000_000:.2f} Bi" if pd.notna(x) and x > 0 else "N/A"
         )

    df_ordenado = df_resultado.sort_values(by=['Setor (brapi)', 'Market Cap Num'], ascending=[True, False])
    
    # Adicionando a coluna 'Tipo' na lista de colunas finais
    colunas_finais = ['Ticker', 'Empresa', 'Setor (brapi)', 'Subsetor (yfinance)', 'País', 'Tipo', 'Market Cap', 'Logo']
    df_final = df_ordenado[colunas_finais]
    
    print("\n--- Relatório Final (Apenas Ações) ---")
    print(df_final.to_string(index=False))

    # --- SALVANDO O ARQUIVO NO CAMINHO ESPECÍFICO ---
    try:
        output_folder = r"E:\Github\finance-manager\datasets"
        output_filename = "scanner_acoes_filtrado.xlsx" # Nome atualizado
        full_path = os.path.join(output_folder, output_filename)
        
        os.makedirs(output_folder, exist_ok=True)
        
        df_final.to_excel(full_path, index=False)
        print(f"\n✅ Resultados salvos com sucesso no caminho: {full_path}")
    except Exception as e:
        print(f"\n❌ Erro ao salvar o arquivo Excel: {e}")
else:
    print("\nNenhuma empresa foi encontrada para os critérios especificados.")

Iniciando scanner final focado (filtrando por tipo 'stock')...
Mapeando ativos dos setores alvo...
✅ Mapeamento concluído. 278 ativos do tipo 'stock' serão analisados.

Iniciando coleta detalhada de dados no yfinance...
Analisando [1/278]: BBDC4.SA...
  -> ✅ INCLUÍDO: BBDC4.SA
Analisando [2/278]: BBAS3.SA...
  -> ✅ INCLUÍDO: BBAS3.SA
Analisando [3/278]: VAMO3.SA...
  -> ✅ INCLUÍDO: VAMO3.SA
Analisando [4/278]: B3SA3.SA...
  -> ✅ INCLUÍDO: B3SA3.SA
Analisando [5/278]: ITSA4.SA...
  -> ✅ INCLUÍDO: ITSA4.SA
Analisando [6/278]: CSAN3.SA...
  -> ✅ INCLUÍDO: CSAN3.SA
Analisando [7/278]: CPLE6.SA...
  -> ✅ INCLUÍDO: CPLE6.SA
Analisando [8/278]: RENT3.SA...
  -> ✅ INCLUÍDO: RENT3.SA
Analisando [9/278]: ITUB4.SA...
  -> ✅ INCLUÍDO: ITUB4.SA
Analisando [10/278]: MOVI3.SA...
  -> ✅ INCLUÍDO: MOVI3.SA
Analisando [11/278]: CMIG4.SA...
  -> ✅ INCLUÍDO: CMIG4.SA
Analisando [12/278]: CPLE3.SA...
  -> ✅ INCLUÍDO: CPLE3.SA
Analisando [13/278]: BBDC3.SA...
  -> ✅ INCLUÍDO: BBDC3.SA
Analisando [14/278]: E

In [30]:
import pandas as pd

df_dados2 = pd.read_excel(r'E:\Github\finance-manager\datasets\scanner_acoes_filtrado.xlsx')

print(df_dados2)

       Ticker                                        Empresa   Setor (brapi)  \
0    VIVT3.SA                         Telefônica Brasil S.A.  Communications   
1    TIMS3.SA                                       TIM S.A.  Communications   
2    FIQE3.SA                 Unifique Telecomunicações S.A.  Communications   
3    BRST3.SA                                            NaN  Communications   
4    TELB3.SA   Telecomunicações Brasileiras S.A. - Telebras  Communications   
..        ...                                            ...             ...   
126  CEBR6.SA         Companhia Energética de Brasília - CEB       Utilities   
127  EMAE4.SA  Empresa Metropolitana de Águas e Energia S.A.       Utilities   
128  AFLT3.SA  Afluente Transmissão de Energia Elétrica S.A.       Utilities   
129  RNEW4.SA                            Renova Energia S.A.       Utilities   
130  RNEW3.SA                            Renova Energia S.A.       Utilities   

                Subsetor (yfinance)    

# Transformação

## Junção das empresas do csv

In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime, date
from dateutil.relativedelta import relativedelta

# --- PASSO 1: LER A LISTA DE ATIVOS DO SEU ARQUIVO ---
arquivo_input = r"E:\Github\finance-manager\datasets\scanner_acoes_filtrado.xlsx"

try:
    print(f"Lendo a lista de ativos do arquivo '{arquivo_input}'...")
    df_input = pd.read_excel(arquivo_input)
    
    # <<< CORREÇÃO APLICADA AQUI >>>
    # Alterado 'Categoria' para 'Setor (brapi)' para bater com o nome real da coluna no arquivo.
    colunas_esperadas = ['Ticker', 'Empresa', 'Setor (brapi)', 'País', 'Market Cap', 'Logo']
    colunas_faltantes = [col for col in colunas_esperadas if col not in df_input.columns]
    
    if colunas_faltantes:
        print("\n" + "="*50)
        print("⚠️ AVISO: Colunas Faltantes no Arquivo de Entrada!")
        print(f"O arquivo '{arquivo_input}' não possui as seguintes colunas: {colunas_faltantes}")
        print("Verifique se você está usando o arquivo correto gerado pelo script anterior.")
        print("="*50 + "\n")

    tickers_para_analisar = df_input['Ticker'].tolist()
    print(f"✅ Sucesso. {len(tickers_para_analisar)} ativos serão analisados.")

except FileNotFoundError:
    print(f"❌ ERRO: O arquivo '{arquivo_input}' não foi encontrado.")
    exit()
except Exception as e:
    print(f"❌ ERRO: Ocorreu um problema ao ler o arquivo: {e}")
    exit()

# --- PASSO 2: BUSCAR OS DADOS DETALHADOS PARA CADA TICKER DA LISTA ---
dados_detalhados_acoes = {}
print("\nIniciando a coleta de dados detalhados...")

for ticker in tickers_para_analisar:
    try:
        print(f"Analisando: {ticker}...")
        stock = yf.Ticker(ticker)
        info = stock.info
        
        info_original = df_input[df_input['Ticker'] == ticker].iloc[0]

        # --- Cálculo da Variação de 5 Anos ---
        variacao_5a = "N/A"
        try:
            end_date = date.today()
            start_date = end_date - relativedelta(years=5)
            hist_5a = stock.history(start=start_date, end=end_date)
            if not hist_5a.empty and len(hist_5a) > 1:
                preco_inicial = hist_5a['Close'].iloc[0]
                preco_final = hist_5a['Close'].iloc[-1]
                if preco_inicial > 0:
                    variacao_5a = ((preco_final / preco_inicial) - 1) * 100
        except Exception:
            variacao_5a = "N/A"

        # Coletando os dados
        dados_detalhados_acoes[ticker] = {
            # <<< CORREÇÃO APLICADA AQUI >>>
            # Lendo da coluna 'Setor (brapi)' e salvando com o mesmo nome para consistência
            "Empresa": info_original.get('Empresa', 'N/A'),
            "Setor (brapi)": info_original.get('Setor (brapi)', 'N/A'), 
            "País": info_original.get('País', 'N/A'),
            "Market Cap (R$)": info_original.get('Market Cap', 'N/A'),
            "Logo": info_original.get('Logo', 'N/A'),
            
            # Dados do yfinance
            "Data Cotação": datetime.fromtimestamp(info.get("regularMarketTime", 0)).strftime('%Y-%m-%d %H:%M:%S'),
            "Preço Atual": info.get("currentPrice", 0),
            "Preço Fechamento": info.get("previousClose", 0),
            "Volume Negociado": info.get("volume", 0),
            "P/L": info.get("trailingPE", "N/A"),
            "P/VP": info.get("priceToBook", "N/A"),
            "Dividend Yield (%)": info.get("dividendYield", 0) * 100,
            "DY 5 Anos Média (%)": info.get("fiveYearAvgDividendYield", "N/A"),
            "Dividendos 12m (R$)": info.get("trailingAnnualDividendRate", "N/A"),
            "ROE (%)": info.get("returnOnEquity", 0) * 100,
            "Margem Líquida (%)": info.get("profitMargins", 0) * 100,
            "Dívida/EBITDA": info.get("debtToEbitda", "N/A"),
            "Variação 12m (%)": info.get("52WeekChange", 0) * 100,
            "Variação 5a (%)": variacao_5a,
        }
        print(f"✅ Dados de {ticker} obtidos com sucesso.")

    except IndexError:
        print(f"❌ ERRO: O ticker '{ticker}' está na sua lista, mas não foi encontrado no arquivo Excel. Pulando.")
    except Exception as e:
        print(f"❌ Não foi possível obter dados para {ticker}. Erro: {e}")

print("\nColeta de dados finalizada.")

# --- PASSO 3: ORGANIZAR E SALVAR O NOVO RELATÓRIO DETALHADO ---
if dados_detalhados_acoes:
    df_resultado_detalhado = pd.DataFrame.from_dict(dados_detalhados_acoes, orient="index")

    colunas_para_formatar = [
        "Preço Atual", "Preço Fechamento", "P/L", "P/VP", "Dividend Yield (%)",
        "DY 5 Anos Média (%)", "Dividendos 12m (R$)", "ROE (%)",
        "Margem Líquida (%)", "Variação 12m (%)", "Variação 5a (%)"
    ]
    for col in colunas_para_formatar:
        if col in df_resultado_detalhado.columns:
            df_resultado_detalhado[col] = df_resultado_detalhado[col].apply(
                lambda x: f"{x:.2f}" if isinstance(x, (int, float)) else x
            )

    print("\n--- Tabela Comparativa de Indicadores Fundamentalistas ---")
    print(df_resultado_detalhado.to_string())

    try:
        df_resultado_detalhado.to_excel("relatorio_completo_b3.xlsx")
        print("\n✅ Relatório detalhado salvo com sucesso no arquivo 'relatorio_completo_b3.xlsx'")
    except Exception as e:
        print(f"\n❌ Erro ao salvar o novo arquivo Excel: {e}")
else:
    print("\nNenhum dado foi coletado.")

Lendo a lista de ativos do arquivo 'E:\Github\finance-manager\datasets\scanner_acoes_filtrado.xlsx'...
✅ Sucesso. 131 ativos serão analisados.

Iniciando a coleta de dados detalhados...
Analisando: VIVT3.SA...
✅ Dados de VIVT3.SA obtidos com sucesso.
Analisando: TIMS3.SA...
✅ Dados de TIMS3.SA obtidos com sucesso.
Analisando: FIQE3.SA...
✅ Dados de FIQE3.SA obtidos com sucesso.
Analisando: BRST3.SA...
✅ Dados de BRST3.SA obtidos com sucesso.
Analisando: TELB3.SA...
✅ Dados de TELB3.SA obtidos com sucesso.
Analisando: TELB4.SA...
✅ Dados de TELB4.SA obtidos com sucesso.
Analisando: DESK3.SA...
✅ Dados de DESK3.SA obtidos com sucesso.
Analisando: OIBR3.SA...
✅ Dados de OIBR3.SA obtidos com sucesso.
Analisando: OIBR4.SA...
✅ Dados de OIBR4.SA obtidos com sucesso.
Analisando: DTCY3.SA...
✅ Dados de DTCY3.SA obtidos com sucesso.
Analisando: ITUB4.SA...
✅ Dados de ITUB4.SA obtidos com sucesso.
Analisando: ITUB3.SA...
✅ Dados de ITUB3.SA obtidos com sucesso.
Analisando: BPAC3.SA...
✅ Dados de